## Load Data

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

data_path = Path('./data/raw')
data_files = list(data_path.glob('*.csv'))
data_files

[PosixPath('data/raw/driver_pts.csv'),
 PosixPath('data/raw/team_price.csv'),
 PosixPath('data/raw/team_pts.csv'),
 PosixPath('data/raw/driver_price.csv')]

In [2]:
frames = {}

for f in data_files:
    frames[f.stem] = pd.read_csv(f, sep='|')


In [3]:
pts_driver = frames.get('driver_pts').copy()
pts_team = frames.get('team_pts').copy()
price_driver = frames.get('driver_price').copy()
price_team = frames.get('team_price').copy()

## Clean and Calculate

### Points Tables

In [4]:
track_cols = [col for col in pts_driver.columns if (len(col) == 3 and col.isupper()) or col.endswith('.1') or col.endswith('.2')]
track_cols

['BAH',
 'SAU',
 'AUS',
 'ITA',
 'USA',
 'SPA',
 'MON',
 'AZE',
 'CAN',
 'BRI',
 'AUS.1',
 'FRA',
 'HUN',
 'BEL',
 'NET',
 'ITA.1',
 'SIN',
 'JAP',
 'USA.1',
 'MEX',
 'BRA',
 'ABU']

#### Tracks with Scores

In [10]:
keep_track_cols = pts_driver[track_cols].sum()[pts_driver[track_cols].sum() > 0].index
keep_track_cols

Index(['BAH', 'SAU', 'AUS', 'ITA', 'USA', 'SPA', 'MON'], dtype='object')

In [11]:
pts_driver[keep_track_cols]

,BAH,SAU,AUS,ITA,USA,SPA,MON
0,34,13,26,5,19,21,11
1,5,45,4,62,56,45,28
2,28,26,33,33,36,33,26
3,4,22,35,44,28,49,46
4,9,-3,-5,2,5,14,19
5,49,41,49,30,50,5,22
6,-4,17,14,13,-1,5,15
7,4,15,-7,24,-5,17,10
8,32,27,-5,7,31,25,34
9,19,-9,-1,30,3,15,-5


In [12]:
pts_driver[keep_track_cols].mean(axis=1)

0     18.428571
1     35.000000
2     30.714286
3     32.571429
4      5.857143
5     35.142857
6      8.428571
7      8.285714
8     21.571429
9      7.428571
10    16.857143
11    20.000000
12    11.857143
13     8.571429
14     4.857143
15     8.142857
16     8.571429
17     0.857143
18     6.142857
19    18.571429
dtype: float64

#### Points Metrics

In [14]:
def calc_metrics(df):

    # df = df.replace(0, np.nan).dropna(axis=1)

    df['avg'] = df[keep_track_cols].mean(axis=1)
    df['max'] = df[keep_track_cols].max(axis=1)
    df['median'] = df[keep_track_cols].median(axis=1)
    
    return df

pts_driver = calc_metrics(pts_driver)
pts_team = calc_metrics(pts_team)


driver_total_avg_points = pts_driver['avg'].sum()  # The total points on average for all drivers
car_total_avg_points = pts_team['avg'].sum()

### Price Tables

In [15]:
def fix_prices(df):
    for col in ['Current Price', 'Season Start PriceSeason Price']: 
        df[col] = df[col].str.replace('$', '').str.replace('m', '').astype(float)

    return df

price_driver = fix_prices(price_driver)
price_team = fix_prices(price_team)

/var/folders/1t/72nlh0w15gzfhdb9xjzgrm1h0000gn/T/ipykernel_64902/3083300969.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col] = df[col].str.replace('$', '').str.replace('m', '').astype(float)


In [16]:
driver_total_price = price_driver['Current Price'].sum()
car_total_price = price_team['Current Price'].sum()

In [17]:
car_total_price

166.10000000000002

In [18]:
price_team['Season Start PriceSeason Price'].sum()

167.5

In [19]:
driver_total_price / (driver_total_price + car_total_price)

0.6267415730337078

In [20]:
driver_total_price / car_total_price

1.6791089704996989

## Performance Prices

Calculate the price a driver or car should fetch based on its actual scoring relative to:

* The total price of all drivers/cars
* Total average points scored by all drivers/cars

In [21]:
pts_driver['pts_price'] = pts_driver['avg'] * driver_total_price / driver_total_avg_points
pts_team['pts_price'] = pts_team['avg'] * car_total_price / car_total_avg_points

pts_driver['pts_price_med'] = pts_driver['median'] * driver_total_price / driver_total_avg_points
pts_team['pts_price_med'] = pts_team['median'] * car_total_price / car_total_avg_points


In [22]:
pts_driver

,Unnamed: 0,Driver,BAH,SAU,AUS,ITA,USA,SPA,MON,AZE,...,USA.1,MEX,BRA,ABU,Total,avg,max,median,pts_price,pts_price_med
0,0,Hamilton Mercedes,34,13,26,5,19,21,11,0,...,0,0,0,0,129,18.428571,34,19.0,16.695174,17.212854
1,1,Verstappen Red Bull,5,45,4,62,56,45,28,0,...,0,0,0,0,245,35.000000,62,45.0,31.707889,40.767285
2,2,Russell Mercedes,28,26,33,33,36,33,26,0,...,0,0,0,0,215,30.714286,36,33.0,27.825290,29.896009
3,3,Perez Red Bull,4,22,35,44,28,49,46,0,...,0,0,0,0,228,32.571429,49,35.0,29.507749,31.707889
4,4,Alonso Alpine,9,-3,-5,2,5,14,19,0,...,0,0,0,0,41,5.857143,19,5.0,5.306218,4.529698
5,5,Leclerc Ferrari,49,41,49,30,50,5,22,0,...,0,0,0,0,246,35.142857,50,41.0,31.837309,37.143527
6,6,Gasly AlphaTauri,-4,17,14,13,-1,5,15,0,...,0,0,0,0,59,8.428571,17,13.0,7.635777,11.777216
7,7,Vettel Aston Martin,4,15,-7,24,-5,17,10,0,...,0,0,0,0,58,8.285714,24,10.0,7.506357,9.059397
8,8,Sainz Ferrari,32,27,-5,7,31,25,34,0,...,0,0,0,0,151,21.571429,34,27.0,19.542413,24.460371
9,9,Tsunoda AlphaTauri,19,-9,-1,30,3,15,-5,0,...,0,0,0,0,52,7.428571,30,3.0,6.729838,2.717819


## Merge

### Driver

* Inconsistent naming, build a match

In [23]:
# Driver Names don't match across tables, grab last name
pts_driver['last_name'] = pts_driver['Driver'].str.split(' ', expand=True)[[0]]

In [24]:
# Cross product, identify which indices contain matching names
index_map = pts_driver['last_name'].apply(lambda y: price_driver['Driver'].str.contains(y).astype(int))
index_map

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
8,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [25]:
driver_map = pd.DataFrame(zip(*np.where(index_map.eq(1))), columns=['pts', 'price'])
driver_map

,pts,price
0,0,1
1,1,0
2,2,2
3,3,4
4,4,10
5,5,3
6,6,8
7,7,11
8,8,5
9,9,15


In [26]:
driver_df = pts_driver.merge(driver_map, left_index=True, right_on='pts').merge(price_driver, left_on='price', right_index=True, suffixes=['', '_price'])
driver_df

,Unnamed: 0,Driver,BAH,SAU,AUS,ITA,USA,SPA,MON,AZE,...,pts_price,pts_price_med,last_name,pts,price,Unnamed: 0_price,Driver_price,Current Price,Season Start PriceSeason Price,Points/Million
0,0,Hamilton Mercedes,34,13,26,5,19,21,11,0,...,16.695174,17.212854,Hamilton,0,1,1,Lewis Hamilton HAM Mercedes,30.1,31.0,0.61
1,1,Verstappen Red Bull,5,45,4,62,56,45,28,0,...,31.707889,40.767285,Verstappen,1,0,0,Max Verstappen VER Red Bull,30.5,30.5,1.15
2,2,Russell Mercedes,28,26,33,33,36,33,26,0,...,27.825290,29.896009,Russell,2,2,2,George Russell RUS Mercedes,23.7,24.0,1.30
3,3,Perez Red Bull,4,22,35,44,28,49,46,0,...,29.507749,31.707889,Perez,3,4,4,Sergio Perez PER Red Bull,18.2,17.5,1.79
4,4,Alonso Alpine,9,-3,-5,2,5,14,19,0,...,5.306218,4.529698,Alonso,4,10,10,Fernando Alonso ALO Alpine,12.2,12.5,0.48
5,5,Leclerc Ferrari,49,41,49,30,50,5,22,0,...,31.837309,37.143527,Leclerc,5,3,3,Charles Leclerc LEC Ferrari,19.0,18.0,1.85
6,6,Gasly AlphaTauri,-4,17,14,13,-1,5,15,0,...,7.635777,11.777216,Gasly,6,8,8,Pierre Gasly GAS AlphaTauri,12.9,13.5,0.65
7,7,Vettel Aston Martin,4,15,-7,24,-5,17,10,0,...,7.506357,9.059397,Vettel,7,11,11,Sebastian Vettel VET Aston Martin,11.4,11.5,0.73
8,8,Sainz Ferrari,32,27,-5,7,31,25,34,0,...,19.542413,24.460371,Sainz,8,5,5,SAI Carlos Sainz Ferrari,17.2,17.0,1.25
9,9,Tsunoda AlphaTauri,19,-9,-1,30,3,15,-5,0,...,6.729838,2.717819,Tsunoda,9,15,15,Yuki Tsunoda TSU AlphaTauri,8.3,8.5,0.90


### Car/Team Join

Straightforward

In [27]:
car_df = pts_team.merge(price_team, on='Team')

In [28]:
car_df

,Unnamed: 0_x,Team,BAH,SAU,AUS,ITA,USA,SPA,MON,AZE,...,Total,avg,max,median,pts_price,pts_price_med,Unnamed: 0_y,Current Price,Season Start PriceSeason Price,Points/Million
0,0,Mercedes,57,34,64,31,40,49,42,0,...,317.0,45.285714,64,42.0,29.614004,27.465354,0,33.7,34.5,1.34
1,1,Red Bull,4,62,39,99,69,94,69,0,...,436.0,62.285714,99,69.0,40.730934,45.121654,1,32.6,32.5,1.91
2,2,Alpine,24,13,10,8,19,33,15,0,...,122.0,17.428571,33,15.0,11.397188,9.809055,4,13.9,14.0,1.25
3,3,Ferrari,76,63,44,30,56,30,51,0,...,350.0,50.000000,76,51.0,32.696850,33.350787,2,25.7,25.0,1.95
4,4,AlphaTauri,10,3,8,36,-3,15,5,0,...,74.0,10.571429,36,8.0,6.913048,5.231496,6,10.0,10.5,1.06
5,5,Aston Martin,14,19,3,32,1,20,15,0,...,104.0,14.857143,32,15.0,9.715636,9.809055,5,10.9,11.5,1.36
6,6,Alfa Romeo,31,3,24,35,3,10,21,0,...,127.0,18.142857,35,21.0,11.864286,13.732677,7,8.8,8.0,2.06
7,7,Williams,15,-1,8,17,26,14,1,0,...,80.0,11.428571,26,14.0,7.473566,9.155118,8,6.5,7.0,1.76
8,8,Haas,27,1,13,8,5,-6,-16,0,...,32.0,4.571429,27,5.0,2.989426,3.269685,9,6.4,6.0,0.71
9,9,Mclaren,9,9,37,41,1,13,26,0,...,136.0,19.428571,41,13.0,12.705062,8.501181,3,17.6,18.5,1.10


## Actual to Performance Price Differentials

In [33]:
driver_df['diff_price'] = driver_df['pts_price'] - driver_df['Current Price']
driver_df[['Driver', 'diff_price', 'Current Price', 'avg', 'median']].sort_values(by='diff_price', ascending=False)

,Driver,diff_price,Current Price,avg,median
5,Leclerc Ferrari,12.837309,19.0,35.142857,41.0
3,Perez Red Bull,11.307749,18.2,32.571429,35.0
11,Bottas Alfa Romeo,8.618794,9.5,20.000000,22.0
2,Russell Mercedes,4.125290,23.7,30.714286,33.0
10,Ocon Alpine,2.871555,12.4,16.857143,20.0
8,Sainz Ferrari,2.342413,17.2,21.571429,27.0
12,Stroll Aston Martin,1.841856,8.9,11.857143,11.0
16,Magnussen Haas,1.665197,6.1,8.571429,6.0
1,Verstappen Red Bull,1.207889,30.5,35.000000,45.0
19,Norris Mclaren,1.024594,15.8,18.571429,22.0


In [30]:
car_df['diff_price'] = car_df['pts_price'] - car_df['Current Price']
car_df[['Team', 'diff_price', 'Current Price', 'avg', 'median']].sort_values(by='avg', ascending=False)

,Team,diff_price,Current Price,avg,median
1,Red Bull,8.130934,32.6,62.285714,69.0
3,Ferrari,6.996850,25.7,50.000000,51.0
0,Mercedes,-4.085996,33.7,45.285714,42.0
9,Mclaren,-4.894938,17.6,19.428571,13.0
6,Alfa Romeo,3.064286,8.8,18.142857,21.0
2,Alpine,-2.502812,13.9,17.428571,15.0
5,Aston Martin,-1.184364,10.9,14.857143,15.0
7,Williams,0.973566,6.5,11.428571,14.0
4,AlphaTauri,-3.086952,10.0,10.571429,8.0
8,Haas,-3.410574,6.4,4.571429,5.0


## Save Data

In [31]:
save_path = Path('./data/transform')

In [32]:
driver_df.to_csv(save_path / 'driver.csv', sep='|', index=False)
car_df.to_csv(save_path / 'car.csv', sep='|', index=False)